In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

We will calculate the average value across components for each mixture.

TODO:
* calculating correlation between elements of mixture1 and mixture2, and calculating the min / max could be usefull.
* data augmentation with fake mixture pairs (same mono, most different mono etc.)

In [4]:
features_mono = pd.read_csv(
    '../results/rekas/mono-descriptors-more_MorganFingerprints_MACCS_and_otherchemfeatures.csv',
    index_col=0
)

In [6]:
train = pd.read_csv('../data/olfaction_mixture/Mixure_Definitions_Training_set.csv')
lead = pd.read_csv('../data/olfaction_mixture/Mixure_Definitions_Leaderboard_set.csv')
test = pd.read_csv('../data/olfaction_mixture/Mixure_Definitions_test_set.csv')

In [7]:
train.index = train['Dataset'] + "_" + train['Mixture Label'].astype(str)
train = train.iloc[:, 2:]
lead.index = lead['Dataset'] + "_" + lead['Mixture Label'].astype(str)
lead = lead.iloc[:, 2:]
test.index = test['Mixture Label'].astype(str)
test = test.iloc[:, 1:]

In [8]:
train.head()

,CID,CID.1,CID.2,CID.3,CID.4,CID.5,CID.6,CID.7,CID.8,CID.9,...,CID.48,CID.49,CID.50,CID.51,CID.52,CID.53,CID.54,CID.55,CID.56,CID.57
Snitz 1_1,6501,264,2879,7685,7731,326,7888,61138,8030,1183,...,0,0,0,0,0,0,0,0,0,0
Snitz 1_2,240,93009,323,8148,7762,3314,460,6184,798,6054,...,0,0,0,0,0,0,0,0,0,0
Snitz 1_3,7710,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Snitz 1_4,31276,93009,11002,323,7966,8148,7632,22201,19310,7762,...,0,0,0,0,0,0,0,0,0,0
Snitz 1_5,10890,93009,11002,6982,323,8797,7966,8148,7632,31252,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def mixture_features(data, features):
    results = pd.DataFrame(index=data.index, columns=features.columns)
    for i in tqdm(data.index):
        cids = data.loc[i]
        ### remove 0, this is placeholder for mixtures with few components
        fil = cids != 0
        ### remove cids with no descriptor data
        cids = list(set(cids) & set(features.index))
        results.loc[i] = features.loc[cids].mean()
    return results

In [10]:
features_train = mixture_features(train, features_mono)
features_lead = mixture_features(lead, features_mono)
features_test = mixture_features(test, features_mono)

100%|██████████| 92/92 [00:00<00:00, 2592.43it/s]


In [72]:
### actually lead mixtures are subset of train mixtures
### so we can ignore lead features

In [11]:
features = pd.concat([features_train, features_test], axis=0)
features.to_csv('../results/features/mix-descriptors-v2.csv')